# 📊 Relatórios Avançados - Imobiliária 3 Irmãos

Este notebook é focado em análise de dados avançada e geração de relatórios executivos para a Imobiliária 3 Irmãos.

## 🎯 Objetivos
- Análise de performance de corretores
- Relatórios de vendas e aluguéis
- Análise de mercado imobiliário
- Dashboards executivos
- Previsões e tendências


In [ ]:
# Importações para análise avançada
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurações
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

print("📊 Ambiente de análise configurado!")
print(f"🕐 {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")


## 📊 Análise de Performance de Corretores

Vamos analisar a performance dos corretores da imobiliária.


In [ ]:
# Configuração da conexão (reutilizar do notebook principal)
from sqlalchemy import create_engine, text
import mysql.connector
from mysql.connector import Error

# Configurações do banco
DB_CONFIG = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'password': 'sua_senha_aqui',  # ⚠️ Altere para sua senha
    'database': 'imobiliaria_3_irmaos',
    'charset': 'utf8mb4'
}

DATABASE_URL = f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

try:
    engine = create_engine(DATABASE_URL, echo=False)
    print("✅ Conexão com banco configurada!")
except Exception as e:
    print(f"❌ Erro na conexão: {e}")


In [ ]:
def generate_executive_report():
    """Gera relatório executivo completo"""
    report = {
        'timestamp': datetime.now().strftime('%d/%m/%Y %H:%M:%S'),
        'summary': {},
        'charts': {}
    }
    
    try:
        with engine.connect() as connection:
            # KPIs principais
            kpis = pd.read_sql("""
                SELECT 
                    (SELECT COUNT(*) FROM usuario) as total_usuarios,
                    (SELECT COUNT(*) FROM imovel) as total_imoveis,
                    (SELECT COUNT(*) FROM corretor) as total_corretores,
                    (SELECT COUNT(*) FROM visita) as total_visitas,
                    (SELECT SUM(valor) FROM imovel) as valor_total_portfolio,
                    (SELECT AVG(valor) FROM imovel) as valor_medio_imoveis
            """, connection)
            
            report['summary'] = kpis.iloc[0].to_dict()
            
            # Performance por corretor
            performance_corretores = pd.read_sql("""
                SELECT 
                    c.id,
                    u.nome as nome_corretor,
                    COUNT(DISTINCT v.id) as total_visitas,
                    COUNT(DISTINCT i.id) as imoveis_atribuidos,
                    COALESCE(SUM(i.valor), 0) as valor_total_imoveis
                FROM corretor c
                JOIN usuario u ON c.fk_usuario_id = u.id
                LEFT JOIN visita v ON c.id = v.fk_corretor_id
                LEFT JOIN imovel i ON c.id = i.fk_corretor_id
                GROUP BY c.id, u.nome
                ORDER BY total_visitas DESC
            """, connection)
            
            report['charts']['performance_corretores'] = performance_corretores
            
            print("✅ Relatório executivo gerado com sucesso!")
            
    except Exception as e:
        print(f"❌ Erro ao gerar relatório: {e}")
    
    return report

# Gerar relatório
executive_report = generate_executive_report()

# Exibir resumo
if executive_report['summary']:
    print("📊 RELATÓRIO EXECUTIVO - IMOBILIÁRIA 3 IRMÃOS")
    print("=" * 60)
    print(f"📅 Gerado em: {executive_report['timestamp']}")
    print(f"👥 Total de Usuários: {executive_report['summary'].get('total_usuarios', 0):,}")
    print(f"🏠 Total de Imóveis: {executive_report['summary'].get('total_imoveis', 0):,}")
    print(f"👨‍💼 Total de Corretores: {executive_report['summary'].get('total_corretores', 0):,}")
    print(f"👀 Total de Visitas: {executive_report['summary'].get('total_visitas', 0):,}")
    print(f"💰 Valor Total do Portfólio: R$ {executive_report['summary'].get('valor_total_portfolio', 0):,.2f}")
    print(f"📈 Valor Médio dos Imóveis: R$ {executive_report['summary'].get('valor_medio_imoveis', 0):,.2f}")
